# ProbNum Quickstart

ProbNum implements probabilistic numerical methods in Python. Such methods quantify _uncertainty arising from finite computation_ or from _stochastic input_.

Below we explain how to get started with ProbNum and its basic functionality.

In [10]:
# Make inline plots vector graphics instead of raster graphics
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

# Plotting
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 18 
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsfonts}', 
                                       r'\usepackage{amsmath}', 
                                       r'\usepackage{bm}']

## Installation

You can install ProbNum using `pip` (or `pip3`).
```bash
pip install probnum
```
If you prefer to use the latest version of ProbNum, install directly from GitHub instead.

```bash
pip install git+https://github.com/probabilistic-numerics/probnum.git
```

## Basic Concepts

The main objects of interest in ProbNum are random variables. `RandomVariable`s have a `Distribution` which models the (numerical) uncertainty on the variable in question.

In [1]:
import numpy as np
import probnum
from probnum.prob import RandomVariable, Normal

np.random.seed(1)

In [2]:
x = RandomVariable(distribution=Normal(0, 1))
print(x.sample())

1.6243453636632417


`RandomVariable`s behave similarly to NumPy arrays and support basic arithmetic, indexing and slicing.

In [3]:
y = 2 * x + 1
print(f"Mean: {y.mean()} and covariance: {y.cov()} of y.")

Mean: 1.0 and covariance: 4.0 of y.


## Probabilistic Numerical Methods

PN methods solve numerical problems (e.g. solution of linear systems, quadrature, differential equations, ...) by treating them as _statistical inference problems_ instead.

At a basic level they can serve as drop-in replacements for classic numerical routines.

In [4]:
# Linear System Ax=b
A = np.array([[7.5, 2.0, 1.0],
              [2.0, 2.0, 0.5],
              [1.0, 0.5, 5.5]])
b = np.array([1, 2, -3])

# Solve using NumPy
x0 = np.linalg.solve(A, b)
print(x0)

# Solve using ProbNum
x1, _, _, info = probnum.linalg.problinsolve(A, b)
print(x1.mean())

[-0.12366738  1.28358209 -0.63965885]
[-0.12366738  1.28358209 -0.63965885]


However, probabilistic numerical methods return random variables instead of just numbers. Their distribution models the uncertainty arising from finite computation or stochastic input.

In [5]:
# Solve with limited computational budget
x1, _, _, _ = probnum.linalg.problinsolve(A, b, maxiter=2)

# Uncertainty in output
print(x1.cov().todense())

# Sample from output random variable
x1.sample()

[[ 2.23355410e-01 -7.52102244e-01  7.23806730e-03]
 [-7.52102244e-01  2.53254571e+00 -2.43726653e-02]
 [ 7.23806730e-03 -2.43726653e-02  2.34557194e-04]]


 /home/j/.virtualenvs/probnum/lib/python3.8/site-packages/probnum/linalg/linearsolvers/matrixbased.py:518: UserWarning:Iteration terminated. Solver reached the maximum number of iterations.


array([ 0.08801172,  0.57079737, -0.63279916])

Here, the probabilistic linear solver has identified one component of the solution already with a high degree of confidence, while there is still some uncertainty about the others left due to the early termination.

### Encoding Prior Knowledge

If we have prior knowledge about the problem setting, we can encode this into a PN method by specifying a prior distribution on the input. For this problem we observe that the matrix $A$ is symmetric. Additionally, suppose we are given an approximate inverse of the system matrix. 

In [6]:
# Approximate inverse of A
Ainv_approx = np.array([[ 0.2  , -0.18, -0.015],
                        [-0.18 ,  0.7 , -0.03 ],
                        [-0.015, -0.03,  0.20 ]])
print(A @ Ainv_approx)

[[1.125  0.02   0.0275]
 [0.0325 1.025  0.01  ]
 [0.0275 0.005  1.07  ]]


We define prior distributions over $A$ and $A^{-1}$. Since there is no stochasticity in the problem definition we choose a highly concentrated symmetric prior over $A$. As a prior over the inverse we use the approximate inverse as a mean for the symmetric normal distribution.

In [7]:
from probnum.linalg.linops import SymmetricKronecker, Identity

# Prior distribution(s)
A0 = RandomVariable(distribution=Normal(mean=A, 
                                        cov=SymmetricKronecker(10 ** -6 * Identity(A.shape[0]))))
Ainv0 = RandomVariable(distribution=Normal(mean=Ainv_approx, 
                                           cov=SymmetricKronecker(0.1 * Identity(A.shape[0]))))

In [8]:
# Solve linear system with prior knowledge
x1, _, _, _ = probnum.linalg.problinsolve(A, b, A0=A0, Ainv0=Ainv0, maxiter=2)

# Uncertainty in output
print(x1.cov().todense())

# Sample from output random variable
x1.sample()

[[0.00037817 0.0015451  0.00064998]
 [0.0015451  0.00631279 0.00265563]
 [0.00064998 0.00265563 0.00111716]]


array([-0.14049052,  1.21484787, -0.66857357])

We observe that after the same number of steps in our algorithm all components are approximately identified and the uncertainty in the output is much lower. 

_Remark:_ The reader familiar with linear solvers might recognize that the prior on the inverse plays a similar role to the preconditioner for classic linear solvers.